<a href="https://colab.research.google.com/github/nelsonbeas33/Datos-Masivos/blob/main/tarea2/tarea2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,031 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,122 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRe

In [3]:
import os
import sys

import findspark
findspark.init()
findspark.find()

from pyspark import SparkContext
from pyspark.sql import SparkSession

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
sc = SparkContext("local", "RDDs")

In [5]:
spark = SparkSession.builder.appName("CSVToSpark").getOrCreate()

In [6]:
df_spark = spark.read.csv('/content/drive/MyDrive/Datos Masivos/order_products__prior.csv', header=True, inferSchema=True)
df_spark.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



In [7]:
df_spark.describe().show()

+-------+------------------+------------------+-----------------+-------------------+
|summary|          order_id|        product_id|add_to_cart_order|          reordered|
+-------+------------------+------------------+-----------------+-------------------+
|  count|          32434489|          32434489|         32434489|           32434489|
|   mean|1710748.5189427834|25576.337535424096|8.351075517175559| 0.5896974667922161|
| stddev|  987300.696452965|14096.689090257634|7.126671161204477|0.49188857672328484|
|    min|                 2|                 1|                1|                  0|
|    max|           3421083|             49688|              145|                  1|
+-------+------------------+------------------+-----------------+-------------------+



In [9]:
rdd = df_spark.rdd
rdd.take(5)
for row in rdd.take(5):  # Muestra las primeras 5 filas
    print(row)

Row(order_id=2, product_id=33120, add_to_cart_order=1, reordered=1)
Row(order_id=2, product_id=28985, add_to_cart_order=2, reordered=1)
Row(order_id=2, product_id=9327, add_to_cart_order=3, reordered=0)
Row(order_id=2, product_id=45918, add_to_cart_order=4, reordered=1)
Row(order_id=2, product_id=30035, add_to_cart_order=5, reordered=0)


In [11]:
#importamos las categoraias
df_categorias = spark.read.csv('/content/drive/MyDrive/Datos Masivos/aisles.csv', header=True, inferSchema=True)
df_categorias.printSchema()

rdd_categorias = df_categorias.rdd
rdd_transformado = rdd_categorias.map(lambda row: row['aisle'].split()[0]) #tomo solo la primera palabra

for row in rdd_transformado.take(5):  # Muestra las primeras 5 filas
    print(row)

root
 |-- aisle_id: integer (nullable = true)
 |-- aisle: string (nullable = true)

prepared
specialty
energy
instant
marinades


In [16]:
#para las ordenes, tomamos solo las que tienen un valor igual a la primera palabra buscada:

rdd_ordenes_filtrado = rdd_transformado.filter(lambda palabra: 'specialty' in palabra.lower())

# Muestra la primera fila filtrada
for row in rdd_ordenes_filtrado.take(1):
    print(row)

specialty


In [17]:
#particionando las ordenes de compra:
rdd_reparticionado = rdd.repartition(4)
rdd_reparticionado.getNumPartitions()

4

In [18]:
#contamos el numero de registros
rdd.count()

32434489